In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor


torch có nhiều module như torchvision, torchtext, torchaudio, tại đây ta dùng torchvision cùng với datasets FashionMNIST của nó  
Bộ dữ liệu FashionMNIST chứa ảnh 28×28 grayscale (đen trắng) của các loại quần áo  
Mỗi mẫu trong training_data là **1 tuple chứa dữ liệu và nhãn**

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", #tải dữ liệu về thư mục data
    train=True,  #dùng tập training data để làm tập huấn luyện (60k ảnh)
    download=True, #nếu chưa có thì tải về
    transform=ToTensor() #chuyển ảnh sang tensor có giá trị trong [0;1] để xử lý
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,  #tập này có 10k dữ liệu, chỉ định là tập test
    download=True, 
    transform=ToTensor(),
)

In [3]:
image, label = training_data[0] #training_data hoạt động tương tự list, chứa các tuple(img, label), img là tensor còn label là int
image.shape, label

(torch.Size([1, 28, 28]), 9)


### ***Dataloader*** : Lấy dữ liệu từ dataset và chia nhỏ thành từng **batch** để mô hình học hiệu quả hơn  
>>>**from torch.utils.data import DataLoader  
>>>train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)**  

Khái niệm về iterable : *là có thể dùng vòng lặp để duyệt qua một iterable*  

Đối với dataloader, nó đóng gói dataset thành 1 iterable, mỗi lần lặp qua 1 batch  

Dataloader hỗ trợ **batching(đóng gói), sampling(phân chia), shuffling (xáo trộn),..**

In [4]:
from torch.utils.data import DataLoader

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
#train_dataloader giờ là 1 iterable chứa 937.5 batch, mỗi batch chứa batch_size = 64 mẫu, mỗi mẫu là 1 tuple (image, label)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(type(X), type(y), y[:10])
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
<class 'torch.Tensor'> <class 'torch.Tensor'> tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7])


### Giải thích về ***Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])***:  
#### **X:** tensor thông tin img
>**N:** batch size  
>**C:** channel (1 là ảnh đen trắng)  
>**H, W:** Height - Weight   

#### **y:** tensor chứa 64 label  
#### **=> thông tin của đoạn code : 1 batch trong test_dataloader chứa 64 mẫu, đen trắng, size 28x28 cùng vector label chứa 64 label**  


### ***TẠO MODEL NN***

In [5]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


##### To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the accelerator such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [6]:
from torch import nn
# Define model
class NeuralNetwork(nn.Module): #tạo class NeuralNetwork kế thừa từ nn.Module, mọi model đều phải kế thừa từ nn.Module
    def __init__(self):
        super().__init__() #khởi tạo constructor của lớp cha cùng lớp con
        self.flatten = nn.Flatten() # định nghĩa lại layer flatten để model biết phải xử lý layer này
        self.linear_relu_stack  = nn.Sequential( #Sequential là hàm trong class nn, cho phép thực hiện các layers bên trong theo tuần tự
                                                 #linear_relu_stack là tên biến
            nn.Linear(28*28, 512), # chuyển ảnh 28x28 đầu vào thành vector 512, có thể hiểu như lọc ra 512 features
            nn.ReLU(),
            nn.Linear(512, 512), # tìm hiểu và học sâu hơn 512 features này
            nn.ReLU(),
            nn.Linear(512, 10) # xuất ra giá trị phân loại (có 10 label)
        )

    def forward(self, x): #hàm đặc biệt của NN, khi gọi model(data) sẽ tương đương với model.foward(data)
        x = self.flatten(x) # sau khi gọi hàm thì ủi dẹt x
        logits = self.linear_relu_stack(x) # sau đó cho x đi qua 5 layer tính toán
        return logits # và trả về

model = NeuralNetwork().to(device) # chuyển việc xử lý class sang gpu
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0300,  0.0011, -0.0039,  ..., -0.0157,  0.0354,  0.0257],
                      [-0.0061,  0.0262,  0.0036,  ..., -0.0269, -0.0340,  0.0353],
                      [-0.0035,  0.0283,  0.0293,  ...,  0.0027,  0.0004,  0.0002],
                      ...,
                      [ 0.0154,  0.0243, -0.0291,  ..., -0.0135, -0.0024,  0.0055],
                      [ 0.0003,  0.0127,  0.0228,  ...,  0.0289, -0.0109,  0.0262],
                      [ 0.0248,  0.0105,  0.0253,  ..., -0.0045, -0.0314,  0.0285]],
                     device='cuda:0')),
             ('linear_relu_stack.0.bias',
              tensor([-1.1580e-02,  2.1740e-02, -6.4930e-03, -1.9777e-02,  2.8955e-02,
                      -3.4130e-02,  2.9732e-02,  1.2633e-02, -1.4167e-02,  9.7271e-03,
                      -3.1604e-02,  3.3140e-02, -8.7146e-03,  1.5209e-02,  2.1522e-02,
                       1.3029e-02,  1.8455e-02, -2.0447e-02,  3.4543e-02,  3.2832

#### ***Khởi tạo Loss Function và Optimization Function***

In [8]:
loss_fn = nn.CrossEntropyLoss() # Khởi tạo loss_fn là CEL, CEL là chuẩn cho bài phân loại nhiều lớp (đa label)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Khởi tạo optimizer là hàm SGD (Stochastic Gradient Descent)

#### ***TẠO HÀM TRAIN LOOP***

In [9]:
batch_size = train_dataloader.batch_size
number_of_batch = len(train_dataloader) 
batch_size, number_of_batch

(64, 938)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer): # 4 tham số của hàm đều đã được định nghĩa ở trên
    size = len(dataloader.dataset)
    model.train() #đặt model vào chế độ huấn luyện
    for batch, (X, y) in enumerate(dataloader): # enumerate : trả về chỉ số và giá trị của iterable
        #tức duyệt qua 938 batch, giá trị mỗi batch được lưu trong X, trả về chỉ số của batch hiện tại, giá trị batch X và label y
        X, y = X.to(device), y.to(device) # X,y là tensor, chuyển về gpu
        # Compute prediction error
        pred = model(X) # huấn luyện trên batch này
        loss = loss_fn(pred, y) # tính loss function

        # Backpropagation có thể xem như mặc định với trình độ này của bé
        loss.backward() # tính gradient của loss
        optimizer.step() # cập nhật dựa trên hàm tối ưu hóa
        optimizer.zero_grad() # đặt gradient của hàm về 0

        if batch % 100 == 0: # mỗi 100 batch thì in ra sai số và batch hiện tại
            # cần để ý vì loss này không tích trữ, tức đây là loss ở batch 100, 200, 300,... đã được tối ưu dần dần
            loss, current = loss.item(), (batch + 1) * len(X) # len(X) là số mẫu chứa trong 1 batch do X là giá trị 1 batch
            #(batch+1)*len(X) là số mẫu đã xử lý được đến hiện tại
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
num_batches = len(test_dataloader)
num_batches

157

In [12]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() #bật c hế độ đánh giá evaluation
    test_loss, correct = 0, 0 # test_loss : tổng loss qua tất cả batch, correct : tổng mẫu đã đoán đúng
    with torch.no_grad(): # tắt tính toán gradient, không cập nhật trọng số w và b
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.short).sum().item()  # pred = số mẫu, xác suất. ở mỗi mẫu, argmax(1) trả về vị trí có xác suất cao nhất, là giá trị nó dự đoán. so sánh với y
            #chuyển true false về 1 0, trả về giá trị = số mẫu đoán đúng 
    test_loss /= num_batches 
    correct /= size #số mẫu đoán đúng trên tổng số mẫu
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
for X,y in train_dataloader:
    print(X.shape, y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295770  [   64/60000]
loss: 2.285049  [ 6464/60000]
loss: 2.263697  [12864/60000]
loss: 2.261185  [19264/60000]
loss: 2.240911  [25664/60000]
loss: 2.210731  [32064/60000]
loss: 2.228424  [38464/60000]
loss: 2.188241  [44864/60000]
loss: 2.186931  [51264/60000]
loss: 2.147897  [57664/60000]
Test Error: 
 Accuracy: 44.1%, Avg loss: 2.142941 

Epoch 2
-------------------------------
loss: 2.163229  [   64/60000]
loss: 2.148820  [ 6464/60000]
loss: 2.096243  [12864/60000]
loss: 2.112169  [19264/60000]
loss: 2.045548  [25664/60000]
loss: 1.992482  [32064/60000]
loss: 2.033658  [38464/60000]
loss: 1.948190  [44864/60000]
loss: 1.959773  [51264/60000]
loss: 1.871831  [57664/60000]
Test Error: 
 Accuracy: 52.3%, Avg loss: 1.872226 

Epoch 3
-------------------------------
loss: 1.921696  [   64/60000]
loss: 1.878871  [ 6464/60000]
loss: 1.773262  [12864/60000]
loss: 1.812886  [19264/60000]
loss: 1.680622  [25664/60000]
loss: 1.648010  [32064/600

#### ***LƯU VÀ TẢI MODEL***

##### **Cách cơ bản dễ dàng nhưng không khuyến khích, tạo file lưu vào thư mục gốc**

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


##### **Cách chuyên nghiệp và thường dùng nhất, tạo folder riêng và lưu vào**  
Lý do chỉ lưu state_dict mà không lưu cả model vì khi ta import có thể gây xung đột class với người khác

In [16]:
from pathlib import Path
MODEL_PATH = Path('MODELS') # tạo thư mục models chứa các models
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = 'Introduction_Model.pth'
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME
print('Save model to {MODEL_SAVE_PATH}')
torch.save(obj = model.state_dict(), f = MODEL_SAVE_PATH)

Save model to {MODEL_SAVE_PATH}


##### **TẢI MODEL**

In [17]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(f = MODEL_SAVE_PATH, weights_only=True))

<All keys matched successfully>

In [18]:
sample = test_data[0] # mỗi mẫu trong test data là 1 tuple chứa img và label
sample_img = sample[0]
sample_label = sample[1]

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

with torch.inference_mode():
    sample_img = sample_img.to(device)
    pred = model(sample_img)
    id = pred.argmax().item()
    print(type(id))
    print('giá trị của pred là xác suất sample nằm trong 1 trong số 10 label')
    print('trong đó vị trí index = 9 tức ankle boot có xác suất cao nhất là 5.4 nên argmax trả về 9')
    predicted, actual = classes[id], classes[sample_label]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

<class 'int'>
giá trị của pred là xác suất sample nằm trong 1 trong số 10 label
trong đó vị trí index = 9 tức ankle boot có xác suất cao nhất là 5.4 nên argmax trả về 9
Predicted: "Ankle boot", Actual: "Ankle boot"
